## Data Cleaning and Cross Validation

In this NB we walk thru cleaning the Titanic data set available from 
[Kaggle](https://www.kaggle.com/c/titanic).  There are two files in this data set: `train.csv` and `test.csv`. Note that observations in the `train.csv` file have a `Survived` column whereas observations in `test.csv` do no. The goal of the Kaggle competition is to use the observations in `train.csv` to develope a classifier which will be used to predict whether a person in the `test.csv` data set survived the Titanic or not.  We will use this data set to demonstrate how to (i) set up an ml pipeline for preprocessing the data and (ii) use cross validation to measure the performance of the classifier

In [ ]:
import pandas as pd

In [ ]:
train = pd.read_csv('train.csv')

### Dealing with missing (NaN) values

The Pandas idiosyncratic way of determining which observations have missing values is:

In [ ]:
train.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [ ]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


Three general approaches to dealing with missing values.  
1. Omit the observation all together
2. Omit just the column (variable) with the missing value
3. "Fill in" the missing value.  A process known as imputation

In [ ]:
train[train.Embarked.isna()]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
61,62,1,1,"Icard, Miss. Amelie",female,38.0,0,0,113572,80.0,B28,NaN
829,830,1,1,"Stone, Mrs. George Nelson (Martha Evelyn)",female,62.0,0,0,113572,80.0,B28,NaN


There isn't an easy way for us to determine where these two passengers Embarked.  So we can either drop this variable or drop the two observations.   Lets do the later.  We can drop these in two ways:

In [ ]:
df = train[train.Embarked.notna()]
df.shape

(889, 12)

In [ ]:
df = train.dropna(subset=['Embarked'])
df.shape

(889, 12)

Now, what about the `Age` and `Cabin` variables which also have NA values?  `Cabin` number doesn't seem to be a strong predictor, so lets omit that.  Lets impute the `Age` with the mean value.

In [ ]:
features = 'Sex Embarked    Fare     Age'.split()
features

['Sex', 'Embarked', 'Fare', 'Age']

In [ ]:
X = df[features]
y = df.Survived

#### Imputation

In [ ]:
X.Age.head(20)

0     22.0
1     38.0
2     26.0
3     35.0
4     35.0
5      NaN
6     54.0
7      2.0
8     27.0
9     14.0
10     4.0
11    58.0
12    20.0
13    39.0
14    14.0
15    55.0
16     2.0
17     NaN
18    31.0
19     NaN
Name: Age, dtype: float64

In [ ]:
X.Age.isna().sum()

177

In [ ]:
from sklearn.impute import SimpleImputer

In [ ]:
imp = SimpleImputer()

In [ ]:
imp.fit(X[['Age']])

SimpleImputer()

In [ ]:
imp.transform(X[['Age']])[:20]

array([[22.       ],
       [38.       ],
       [26.       ],
       [35.       ],
       [35.       ],
       [29.6420927],
       [54.       ],
       [ 2.       ],
       [27.       ],
       [14.       ],
       [ 4.       ],
       [58.       ],
       [20.       ],
       [39.       ],
       [14.       ],
       [55.       ],
       [ 2.       ],
       [29.6420927],
       [31.       ],
       [29.6420927]])

In [ ]:
imp.fit_transform(X[['Age']]);  

**Important:** Make sure you understand the difference between `.fit`, `.transform`, and `.fit_transform`

### One Hot Encoding

Many ML algorithms require the feature matrix to be numeric.  This will require nominal variables to be encoded with One Hot Encoding

In [1]:
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder

In [2]:
ohe = OneHotEncoder(sparse=False)  # false is just for illustration

In [3]:
ohe.fit_transform(X[['Embarked']])

NameError: ignored

## Baseline

In [ ]:
df.Survived.value_counts(normalize=True)

0    0.617548
1    0.382452
Name: Survived, dtype: float64

## Pipeline

In [ ]:
from sklearn.compose import make_column_transformer

In [ ]:
imp = SimpleImputer()
ohe = OneHotEncoder()

ct = make_column_transformer(
       (ohe, ['Sex', 'Embarked']),
       (imp, ['Age']),
       remainder = 'passthrough'
)

In [ ]:
a = ct.fit_transform(X)
a[:5, : ]

array([[ 0.    ,  1.    ,  0.    ,  0.    ,  1.    , 22.    ,  7.25  ],
       [ 1.    ,  0.    ,  1.    ,  0.    ,  0.    , 38.    , 71.2833],
       [ 1.    ,  0.    ,  0.    ,  0.    ,  1.    , 26.    ,  7.925 ],
       [ 1.    ,  0.    ,  0.    ,  0.    ,  1.    , 35.    , 53.1   ],
       [ 0.    ,  1.    ,  0.    ,  0.    ,  1.    , 35.    ,  8.05  ]])

The above is the transformation of the below data frame

In [ ]:
X.head(5)

,Sex,Embarked,Fare,Age
0,male,S,7.2500,22.0
1,female,C,71.2833,38.0
2,female,S,7.9250,26.0
3,female,S,53.1000,35.0
4,male,S,8.0500,35.0


## Cross Validation

In [ ]:
from sklearn.pipeline import make_pipeline

In [ ]:
ct = make_column_transformer(
       (ohe, ['Sex', 'Embarked']),
       (imp, ['Age']),
       remainder = 'passthrough'
)

from sklearn.linear_model import LogisticRegression

lgr = LogisticRegression()

p1 = make_pipeline(ct, lgr)

X = df[features]
y = df.Survived


from sklearn.model_selection import cross_val_score

cross_val_score( p1, X , y, cv=5, scoring='accuracy').mean()



0.7795404050022217

In [5]:
!ls

sample_data  test.csv  train.csv


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!ls

MyDrive  Shareddrives


In [ ]:
cd MyDrive

/content/drive/MyDrive


In [ ]:
!ls

 00-a5-rel-model-normalization-rubric-f21.xlsx
 01-1-ds-s17-introduction.pptx
 2017-madurai-readers-club-data-science.pptx
 20190507_172647.jpg
 20190507_172658.jpg
 20190507_172709.jpg
 373
 582a677208ae138f1bf34cba.pdf
 5-morning-rituals-that-help-me-win-the-day1.pdf
 67262-f16-schedule-landscape.gdoc
 67262-f19-syllabus.pdf
'67475 Teams 2016.xlsx'
'67475 Teams 2016.xlsx.gsheet'
 784758362150.pdf
 95885-project-2-s20.pdf
'95885-s20-syllabus (1).pdf'
 95885-s20-syllabus.pdf
 a1-jaiswalshubham.xlsx.gsheet
'a1_load (1).sql'
 a1_load.sql
'a2-assignment_johannas (3).Rmd'
 a3-flight-connections-exercise.ipynb
 a3-flight-connections-solution.ipynb
 a3-gender-neutral-babynames-exercise.ipynb
 a3-gender-neutral-babynames-solution.ipynb
 a3-joins
'a3-visualization-critique-ds-s20 (1).pdf'
 a3-visualization-critique-ds-s20.pdf
 a4-charleyz.xlsx.gsheet
 a5-convert-to-relational-model.docx
 a5-recommender-systems-template.pdf
'a6-mapreduce-mrjob-ds-s20 (1).pdf'
 a6-mapreduce-mrjob-ds-s20.pdf
'a7-

In [ ]:
!pwd

/content/drive/MyDrive


In [ ]:
cd ..

/content/drive


In [ ]:
ls

MyDrive/  Shareddrives/


In [ ]:
cd Shareddrives/

/content/drive/Shareddrives


In [ ]:
ls

'Jigsaw: Identifying toxicity and bias online'/
